In [2]:
import os
os.getcwd()


'/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/notebooks'

In [4]:
import os
os.listdir("../data_processed")


['R6jisatsunojoukyou_page32_table1_cleaned.csv',
 'master_suicide_data_aggregated.csv']

In [5]:
import pandas as pd

df = pd.read_csv("../data_processed/R6jisatsunojoukyou_page32_table1_cleaned.csv")
df.head()


,年,年齢層,人数
0,昭和53年,～９歳,866
1,昭和54年,～９歳,919
2,昭和55年,～９歳,678
3,昭和56年,～９歳,620
4,昭和57年,～９歳,599


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   年       517 non-null    object
 1   年齢層     517 non-null    object
 2   人数      428 non-null    object
dtypes: object(3)
memory usage: 12.2+ KB


In [8]:
import re

def era_to_year(val):
    # Extract era and number
    m = re.match(r"(令和|平成|昭和)(\d+)年", val)
    if not m:
        return None
    era, num = m.group(1), int(m.group(2))
    
    if era == "令和":
        return 2019 + (num - 1)
    if era == "平成":
        return 1989 + (num - 1)
    if era == "昭和":
        return 1926 + (num - 1)

df["year"] = df["年"].apply(era_to_year)
df[["年", "year"]].head()


,年,year
0,昭和53年,1978.0
1,昭和54年,1979.0
2,昭和55年,1980.0
3,昭和56年,1981.0
4,昭和57年,1982.0


In [9]:
df["年齢層"] = (
    df["年齢層"]
    .str.replace(" ", "", regex=False)
    .str.replace("　", "", regex=False)
    .str.replace("合計", "合計")
    .str.replace("合計", "合計")
    .str.replace("合計", "合計")
)

df["年齢層"].unique()


array(['～９歳', '10～19歳', '20～29歳', '30～39歳', '40～49歳', '50～59歳', '60～69歳',
       '70～79歳', '80歳～', '不詳', '合計'], dtype=object)

In [12]:
df["年齢層"] = df["年齢層"].str.replace("合計", "合計").str.replace("合", "合").str.replace("計", "計")
df["年齢層"] = df["年齢層"].str.replace("合計", "合計")


In [13]:
df["人数"] = (
    df["人数"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .astype(float)
)


In [14]:
df.sample(10)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   年       517 non-null    object 
 1   年齢層     517 non-null    object 
 2   人数      428 non-null    float64
 3   year    495 non-null    float64
dtypes: float64(2), object(2)
memory usage: 16.3+ KB


In [15]:
df["年齢層"].unique()
df.sample(10)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   年       517 non-null    object 
 1   年齢層     517 non-null    object 
 2   人数      428 non-null    float64
 3   year    495 non-null    float64
dtypes: float64(2), object(2)
memory usage: 16.3+ KB


In [16]:
# 1. Show 年 values that failed conversion
df[df["year"].isna()][["年"]].drop_duplicates()

# 2. Show unique 年齢層 values
df["年齢層"].unique()

# 3. Show 人数 values that could not convert
bad = df[df["人数"].isna()][["年", "年齢層", "人数"]]
bad.head(20)


,年,年齢層,人数
47,昭和53年,10～19歳,NaN
48,昭和54年,10～19歳,NaN
49,昭和55年,10～19歳,NaN
50,昭和56年,10～19歳,NaN
51,昭和57年,10～19歳,NaN
52,昭和58年,10～19歳,NaN
53,昭和59年,10～19歳,NaN
54,昭和60年,10～19歳,NaN
55,昭和61年,10～19歳,NaN
56,昭和62年,10～19歳,NaN


In [17]:
df[df["year"].isna()]["年"].drop_duplicates().tolist()


['平成元年', '令和元年']

In [18]:
df["年齢層"].drop_duplicates().tolist()


['～９歳',
 '10～19歳',
 '20～29歳',
 '30～39歳',
 '40～49歳',
 '50～59歳',
 '60～69歳',
 '70～79歳',
 '80歳～',
 '不詳',
 '合計']

In [19]:
df.loc[47:70, ["人数"]]


,人数
47,NaN
48,NaN
49,NaN
50,NaN
51,NaN
52,NaN
53,NaN
54,NaN
55,NaN
56,NaN


In [21]:
# Fix era conversion properly

import re

def convert_japanese_era_to_year(era_str):
    # Normalize full-width digits
    era_str = era_str.replace("元年", "1年")

    patterns = {
        "令和": 2018,
        "平成": 1988,
        "昭和": 1925,
        "大正": 1911,
        "明治": 1867
    }

    for era, base in patterns.items():
        if era in era_str:
            m = re.search(r"(\d+)年", era_str)
            if m:
                year_num = int(m.group(1))
                return base + year_num
    return None


In [22]:
df["year"] = df["年"].apply(convert_japanese_era_to_year)


In [23]:
df["年齢層"].drop_duplicates().tolist()


['～９歳',
 '10～19歳',
 '20～29歳',
 '30～39歳',
 '40～49歳',
 '50～59歳',
 '60～69歳',
 '70～79歳',
 '80歳～',
 '不詳',
 '合計']

In [24]:
age_map = {
    "～９歳": "0～9歳",
    "10～19歳": "10～19歳",
    "20～29歳": "20～29歳",
    "30～39歳": "30～39歳",
    "40～49歳": "40～49歳",
    "50～59歳": "50～59歳",
    "60～69歳": "60～69歳",
    "70～79歳": "70～79歳",
    "80歳～": "80歳以上",
    "不詳": "不詳",
    "合計": "合計"
}

df["年齢層"] = df["年齢層"].str.strip().replace(age_map)


In [25]:
df.loc[47:70, ["人数"]]


,人数
47,NaN
48,NaN
49,NaN
50,NaN
51,NaN
52,NaN
53,NaN
54,NaN
55,NaN
56,NaN


In [42]:
## Load more tables

In [41]:
import pandas as pd
import glob

path = "/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed"

files = glob.glob(f"{path}/*.csv")
files


['/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed/R6jisatsunojoukyou_page32_table1_cleaned.csv',
 '/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed/page31_cleaned.csv',
 '/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed/page33_cleaned.csv',
 '/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed/page30_table3_cleaned.csv',
 '/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed/page30_table4_cleaned.csv']

In [44]:
import pandas as pd
import glob
import os

path = "/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed"

files = glob.glob(f"{path}/*cleaned.csv")

datasets = {}

for f in files:
    name = os.path.basename(f).replace(".csv", "")
    datasets[name] = pd.read_csv(f)

datasets


{'R6jisatsunojoukyou_page32_table1_cleaned':          年  年齢層      人数
 0    昭和53年  ～９歳     866
 1    昭和54年  ～９歳     919
 2    昭和55年  ～９歳     678
 3    昭和56年  ～９歳     620
 4    昭和57年  ～９歳     599
 ..     ...  ...     ...
 512   令和２年  合 計  21,081
 513   令和３年  合 計  21,007
 514   令和４年  合 計  21,881
 515   令和５年  合 計  21,837
 516   令和６年  合 計  20,320
 
 [517 rows x 3 columns],
 'page31_cleaned':            年  category   value
 0    区分\n年次別         1     自殺者
 1        NaN         1      総数
 2      昭和53年         1  20,788
 3      昭和54年         1  21,503
 4      昭和55年         1  21,048
 ..       ...       ...     ...
 289     令和２年         6    10.8
 290     令和３年         6    11.0
 291     令和４年         6    11.1
 292     令和５年         6    10.9
 293     令和６年         6    10.3
 
 [294 rows x 3 columns],
 'page33_cleaned':        年  category                                              value
 0    NaN         1                                                NaN
 1    NaN         1                     

In [45]:
datasets["page33_cleaned"].head()


,年,category,value
0,NaN,1,NaN
1,NaN,1,NaN
2,自殺者,1,NaN
3,NaN,1,NaN
4,NaN,1,管理的職業従事者\n議員・知事・課長以上の公務員\n会社・公団等の役員\n会社・公団等の部・...


In [46]:
for name, df in datasets.items():
    print(name, df.shape)

R6jisatsunojoukyou_page32_table1_cleaned (517, 3)
page31_cleaned (294, 3)
page33_cleaned (300, 3)
page30_table3_cleaned (49, 3)
page30_table4_cleaned (21, 3)


In [47]:
import pandas as pd

raw31 = pd.read_csv("/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_raw/csvs/R6jisatsunojoukyou_page31_table1.csv")
raw31.head(20)


,0,1,2,3,4,5,6
0,区分\n年次別,自殺者,NaN,NaN,自殺死亡率,NaN,NaN
1,NaN,総数,男性,女性,総数,男性,女性
2,昭和53年,"20,788","12,859","7,929",18.0,22.7,13.6
3,昭和54年,"21,503","13,386","8,117",18.5,23.4,13.8
4,昭和55年,"21,048","13,155","7,893",18.0,22.8,13.3
5,昭和56年,"20,434","12,942","7,492",17.3,22.3,12.5
6,昭和57年,"21,228","13,654","7,574",17.9,23.4,12.6
7,昭和58年,"25,202","17,116","8,086",21.1,29.1,13.3
8,昭和59年,"24,596","16,508","8,088",20.4,27.9,13.2
9,昭和60年,"23,599","15,624","7,975",19.5,26.3,13.0


In [48]:
import pandas as pd
import re

def convert_japanese_year(s):
    mapping = {
        "昭和": 1925,
        "平成": 1988,
        "令和": 2018
    }
    m = re.match(r"(昭和|平成|令和)(\d+|元)年", s)
    if not m:
        return None
    era, num = m.groups()
    y = 1 if num == "元" else int(num)
    return mapping[era] + y

# Load raw
df = pd.read_csv(
    "/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_raw/csvs/R6jisatsunojoukyou_page31_table1.csv"
)

# Fix header manually
df.columns = [
    "年",
    "自殺者_総数",
    "自殺者_男性",
    "自殺者_女性",
    "死亡率_総数",
    "死亡率_男性",
    "死亡率_女性"
]

# Drop row 0 and 1 (they are header rows)
df = df.iloc[2:].reset_index(drop=True)

# Convert numbers with commas
num_cols = [
    "自殺者_総数", "自殺者_男性", "自殺者_女性",
    "死亡率_総数", "死亡率_男性", "死亡率_女性"
]

for c in num_cols:
    df[c] = df[c].astype(str).str.replace(",", "").astype(float)

# Add Gregorian year
df["year"] = df["年"].apply(convert_japanese_year)

df.tail()


,年,自殺者_総数,自殺者_男性,自殺者_女性,死亡率_総数,死亡率_男性,死亡率_女性,year
42,令和２年,21081.0,14055.0,7026.0,16.7,22.9,10.8,2020
43,令和３年,21007.0,13939.0,7068.0,16.7,22.8,11.0,2021
44,令和４年,21881.0,14746.0,7135.0,17.5,24.3,11.1,2022
45,令和５年,21837.0,14862.0,6975.0,17.6,24.6,10.9,2023
46,令和６年,20320.0,13801.0,6519.0,16.4,22.9,10.3,2024


In [52]:

output_path = "/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_clean/gender_cleaned.csv"

df.to_csv(output_path, index=False, encoding="utf-8-sig")
print("Saved:", output_path)



Saved: /home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_clean/gender_cleaned.csv


In [54]:
import pandas as pd

raw30 = pd.read_csv("/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed/page30_table3_cleaned.csv")
raw30.head(20)


,年,問題分類,人数
0,平成27年,家庭問題,"3,641"
1,平成28年,家庭問題,"3,337"
2,平成29年,家庭問題,"3,179"
3,平成30年,家庭問題,"3,147"
4,令和元年,家庭問題,"3,039"
5,令和２年,家庭問題,"3,128"
6,令和３年,家庭問題,"3,200"
7,平成27年,健康問題,"12,145"
8,平成28年,健康問題,"11,014"
9,平成29年,健康問題,"10,778"


In [57]:
df = raw30.copy()

# Normalize category labels
df["問題分類"] = df["問題分類"].str.replace("\n", "", regex=False)

# Fix comma numbers
df["人数"] = df["人数"].astype(str).str.replace(",", "", regex=False).astype(int)

# Map of eras
era_map = {
    "平成": 1988,   # Heisei N = 1988 + N
    "令和": 2018    # Reiwa N = 2018 + N
}

def convert_era(year_str):
    if not isinstance(year_str, str):
        return None

    year_str = year_str.strip()

    # Fix full-width digits → half-width
    year_str = year_str.translate(str.maketrans("０１２３４５６７８９", "0123456789"))

    for era, base in era_map.items():
        if era in year_str:
            num = year_str.replace(era, "").replace("年", "")

            # Handle 元年 → 1
            if num == "元":
                num = "1"

            # Convert
            try:
                return base + int(num)
            except ValueError:
                return None

    return None

df["year"] = df["年"].apply(convert_era)

df.head(20)


,年,問題分類,人数,year
0,平成27年,家庭問題,3641,2015
1,平成28年,家庭問題,3337,2016
2,平成29年,家庭問題,3179,2017
3,平成30年,家庭問題,3147,2018
4,令和元年,家庭問題,3039,2019
5,令和２年,家庭問題,3128,2020
6,令和３年,家庭問題,3200,2021
7,平成27年,健康問題,12145,2015
8,平成28年,健康問題,11014,2016
9,平成29年,健康問題,10778,2017


In [58]:

output_path = "/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_clean/reason_cleaned.csv"

df.to_csv(output_path, index=False, encoding="utf-8-sig")
print("Saved:", output_path)


Saved: /home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_clean/reason_cleaned.csv


In [59]:
import pandas as pd
from pathlib import Path

base = Path("/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_clean")

age_path = base / "age_cleaned.csv"
gender_path = base / "gender_cleaned.csv"
reason_path = base / "reason_cleaned.csv"

age_df = pd.read_csv(age_path)
gender_df = pd.read_csv(gender_path)
reason_df = pd.read_csv(reason_path)

# Validate the key
assert "year" in age_df.columns, "year missing in age_cleaned.csv"
assert "year" in gender_df.columns, "year missing in gender_cleaned.csv"
assert "year" in reason_df.columns, "year missing in reason_cleaned.csv"

# Merge
merged_df = (
    age_df
    .merge(gender_df, on="year", how="outer", suffixes=("_age", "_gender"))
    .merge(reason_df, on="year", how="outer")
)

merged_df.head()


,年_age,年齢層,人数_x,year,年_gender,自殺者_総数,自殺者_男性,自殺者_女性,死亡率_総数,死亡率_男性,死亡率_女性,年,問題分類,人数_y
0,昭和53年,0～9歳,866.0,1978,昭和53年,20788.0,12859.0,7929.0,18.0,22.7,13.6,NaN,NaN,NaN
1,昭和53年,10～19歳,NaN,1978,昭和53年,20788.0,12859.0,7929.0,18.0,22.7,13.6,NaN,NaN,NaN
2,昭和53年,20～29歳,3741.0,1978,昭和53年,20788.0,12859.0,7929.0,18.0,22.7,13.6,NaN,NaN,NaN
3,昭和53年,30～39歳,3597.0,1978,昭和53年,20788.0,12859.0,7929.0,18.0,22.7,13.6,NaN,NaN,NaN
4,昭和53年,40～49歳,3641.0,1978,昭和53年,20788.0,12859.0,7929.0,18.0,22.7,13.6,NaN,NaN,NaN


In [60]:
out_path = base / "master_cleaned_dataset.csv"
merged_df.to_csv(out_path, index=False)
out_path


PosixPath('/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_clean/master_cleaned_dataset.csv')